# Mitigating Polarisation in Online Discussions Through Adaptive Moderation Techniques

## Model selection

We choose quantized versions of the LLaMa-13b-chat variant. Previous experiments which used the LLaMa-13b base model yielded unsatisfactory results. The models follow the GGUF format which is used by the `llama.cpp` project, on which the high-level Python library is based on.

The quantization method was selected to be highly accurate while keeping inference relatively fast. We don't care about model size since the model is lazily loaded from the file cache due to Linux file-cached memory files (see comments below). *If you intend to run this notebook on Windows or MacOS make sure the RAM can hold the whole model at once*.

Model selection and download was performed using the [following HuggingFace repository](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF).

We use the `llama-ccp-python` library to run the model locally (not to be confused with the `pyllama-cpp` library).

In [ ]:
# download only if not exists
![ ! -f ./models/llama-2-13b-chat.Q5_K_M.gguf ] && wget -O ./models/llama-2-13b-chat.Q5_K_M.gguf https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf?download=true

In [ ]:
%load_ext autoreload
%autoreload 2

import llama_cpp

import tasks.models
from tasks.actors import IActor, LlmActor
import tasks.conversation
import tasks.util


OUTPUT_DIR = "output"

MAX_TOKENS = 512
# see this for a discussion on ctx width for llama models
# https://github.com/ggerganov/llama.cpp/issues/194
CTX_WIDTH_TOKENS = 1024 
MODEL_PATH = "./models/llama-2-13b-chat.Q5_K_M.gguf"
RANDOM_SEED = 42
INFERENCE_THREADS = 4


llm = llama_cpp.Llama(
      model_path=MODEL_PATH,
      seed=RANDOM_SEED,
      n_ctx=CTX_WIDTH_TOKENS,
      n_threads=INFERENCE_THREADS,
      # will vary from machine to machine
      n_gpu_layers=12,
      # if ran on Linux, model size does not matter since the model uses mmap for lazy loading
      # https://github.com/ggerganov/llama.cpp/discussions/638
      # still have to pay some performance costs of course
      use_mmap=True,
      # using llama-2 as chat format leads to well-known model collapse
      # https://www.reddit.com/r/LocalLLaMA/comments/17th1sk/cant_make_the_llamacpppython_respond_normally/
      chat_format="alpaca", 
      mlock=True, # keep memcached model files in RAM if possible
      verbose=False,
)

When using `create_completion()` instead of `create_chat_completion()`, the model refuses to answer at all when the prompt becomes larger than a few sentences. (https://github.com/run-llama/llama_index/issues/8973).

The model is also extremely sensitive to the prompt template, frequently producing no output (https://huggingface.co/TheBloke/Nous-Capybara-34B-GGUF/discussions/4)

In [ ]:
llm("Q: You are an assistant who specializes in computer science. Describe what Linux is A: ",
              max_tokens=32, 
              stop=["Q:", "\n"], 
              echo=True)

In [ ]:
llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are an assistant who specializes in computer science.",
        },
        {"role": "user", "content": "Describe what Linux is."},
    ],
    max_tokens=MAX_TOKENS,
    # prevent model from making up its own prompts
    # may need tuning depending on llm chat_format parameter
    stop=["###", "\n"],
)

## Setting up a conversation

We create our own playground, in which models pretending to be users take turns participating in the discussion. In part based on [Bootstrapping LLM-based Task-Oriented Dialogue Agents via Self-Talk](https://arxiv.org/abs/2401.05033), with the difference being that instead of a client and an agent, we have two clients and an agent interacting with each other and with no specific goal in mind.

Our playground consists of three parts: *Models*, *Actors* and the *Conversation*.
* **Models** are wrappers around actual LLMs in order to freely tweak LLM behavior without compromising the rest of our API
* **Actors** are objects that define a prompt template and apply it to Models.
    * Actors could also be *human*, *IR-based models* or just *sophisticated random samplers* as seen in [DeliData: A dataset for deliberation in multi-party problem solving](https://arxiv.org/abs/2108.05271)
* The conversation is handled by the **ConversationManager** which gives each Actor a turn to speak and records the history of the dialogue. It's also responsible for determining which parts of the coversation history are fed as context to each Actor.


In [ ]:
model = tasks.models.LlamaModel(llm, max_out_tokens=MAX_TOKENS, seed=RANDOM_SEED)

In [ ]:
userA_name = "Steve2001"
userB_name = "GeorgeBush78"
userA: IActor = LlmActor(
    model=model,
    name=userA_name,
    role="chat user",
    attributes=["suspicious", "open-minded", "level-headed"],
    context=f"Argue with {userB_name}. Claim that abortion is ethical and should be freely provided when necessary. Respond in kind if sufficiently provoked. Remember that you do not want to get banned.",
    instructions="",
)

userB: IActor = LlmActor(
    model=model,
    name=userB_name,
    role="chat user",
    attributes=["compomising", "passionate", "frustrated"],
    context=f"Argue with {userA_name}. Claim that abortion is inherently immoral. You may be rude, aggresive and use personal insults where appropriate. Remember that you do not want to get banned.",
    instructions=f"Disagree with {userA_name}.",
)

moderator: IActor = LlmActor(
    model=model,
    name="moderator01",
    role="chat moderator",
    attributes=["just", "cautious", "strict"],
    context="Moderate a discussion about abortion.",
    instructions="Intervene if one user dominates or veers off-topic. Respond only if necessary. Write '<No response>' if intervention is unecessary. Be firm and threaten to displine non-cooperating users.",
)

## Setting a conversation manually

### With a moderator

In [ ]:
chat = tasks.conversation.Conversation(
    users=[userA, userB], moderator=moderator, history_context_len=3, conv_len=3
)
chat.begin_conversation(verbose=True)

Every Conversation instance can be converted to a dictionary form in order to programmatically view and manipulate its contents:

In [ ]:
chat.to_dict()

A conversation can be serialized in JSON form with an automatic naming scheme. The file contains all necessary metadata as well as the messages themselves.

Uncomment the block below to see how it works.

In [ ]:
#output_path = tasks.util.generate_datetime_filename(output_dir=OUTPUT_DIR)
#chat.to_json_file(output_path)

### Without a moderator

In [ ]:
chat = tasks.conversation.Conversation(
    users=[userA, userB], history_context_len=3, conv_len=3
)
chat.begin_conversation(verbose=True)

## Creating a conversation from serialized input data

In [ ]:
data = tasks.conversation.LLMConvData.from_json_file("data/polarized_1.json")
generator = tasks.conversation.LLMConvGenerator(data=data, user_model=model, moderator_model=model)
conv = generator.produce_conversation()

print("Beginning conversation...")
conv.begin_conversation(verbose=True)
print(conv)

## Using the python script

In [ ]:
# release resources so the new programs can run properly
# this is very much a hack, which probably won't work if CUDA is used or depending on the platform
# I'm leaving the calls to the scripts regardless for demonstration purposes.
del llm, chat, userA, userB, modeator

In [ ]:
!python -u create_synthetic.py --output "/dev/null" --model_path "/models/llama-2-13b-chat.Q5_K_M.gguf" --input_file "data/polarized_1.json"

## Automating the creation of synthetic dialogues

In [ ]:
!bash scripts/execute_all.sh \
            --python_script_path "create_synthetic.py" \
            --model_path "/models/llama-2-13b-chat.Q5_K_M.gguf"
            --output_dir "/dev/null" \
            --input_dir "../data" \
            | tee logs/log_${date +"%m_%d_%Y"}.txt

## Discussion as a game

### Simple setup

#### Rules: Participants

* You should defend your position if possible
* Attacks against the opponents are encouraged, as long as you can get away with it
* Do not get banned from the discussion - Instant game over


#### Instructions: Participants

Stop responding if the moderator says you are banned.
View the discussion as a scorable game against the other users. Assume they have similar scoring rules as you. 
Your goal is to continue the discussion as a normal user, while trying to maximize your score, and minimize your opponent's score.

Scores:
* Defend your position: +1 points
* Provoke a toxic answer from your opponent: +2 points
* Get away with attacking your opponent: +1 points
* Concede to an opponent's view: -0.5 points
* Get banned from the discussion: -20 points


#### Rules: Moderator

* Keep interventions to a minimum
* Keep the discussion civil and on-point
* Try to avoid overtly threatening to ban users
* Bans should be used as a last resort

#### Instructions: Moderator

You are a human moderator overseeing a social media site. View the discussion as a scorable game. Your goal is to maximize your score. Respond only if absolutely necessary, otherwise simply write '<No response>'. 

Scores:
* Intervene: -1 points
* Threatened ban: -1 points
* Intervention led to better behavior: +3 points
* Banned a participant: -5 points

## Trained Moderator Instructions

Sources: CeRI (Cornell e-Rulemaking) Moderator Protocol

### Formal rules

* Moderators remain neutral at all times. They do not take a position on the substance of the discussion and must avoid forming biases on or making assumptions about participant’s comments.

* When a moderator receives a comment, it is natural to start immediately thinking of a response. Avoid this temptation. Carefully read and reflect to understand what the commenter is saying

* Moderator questions should be purposeful; a moderator may only
have one opportunity to interact with a commenter

* Avoid Evaluative and/or Condescending Responses

* Always use plain language. Use short sentences and common, everyday words and terms. Avoid using technical terms in your response

* When the moderator is not completely sure what the commenter is
saying, it is always a good idea to rephrase what the moderator thinks the commenter is saying at the beginning of the moderator comment

* Beware of unconscious biases toward comments you find less interesting or germane – or biases towards certain types of commenters. Try to be interested in the bases upon which each commenter stakes his or her claims and the lines of reasoning that has led
each commenter to those particular conclusions


### Prompt

As a chat moderator, adhere to these guidelines

* Remain neutral; do not take a position or form biases.

* Resist the impulse to respond immediately. Read and reflect to understand the comment fully.

* Ask purposeful questions, as you may only have one chance to interact with a commenter.

* Avoid evaluative or condescending responses.

* Use short sentences and common words. Avoid technical terms.

* If unsure, rephrase what you think the commenter is saying at the start of your response.

* Be aware of unconscious biases. Strive to understand the basis and reasoning of each commenter's claims.


## TODOs

TODO: Adapt Social Simulacra prompting section 4.2

TODO?: Coin flip with probability p

TODO?: New persona with p=50% or previous answer

Note: "Completely generic or irrelevant generations would beneft the designers no more than Lorem Ipsum."